In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
from scipy.stats import gaussian_kde
from pydoc import help
from scipy.stats.stats import pearsonr
from sklearn.metrics import explained_variance_score
from scipy.stats import wilcoxon
import warnings
import os
import glob
warnings.filterwarnings('ignore')
from __future__ import print_function, division

Populating the interactive namespace from numpy and matplotlib


In [2]:
# change the variables to the parameter being modelled

In [3]:
hpar = 'h99'
hpar2 = 'perc99_1'

In [4]:
direc = './'

# empty list to store the list of csv files
filelist =[]

# empty lists to put the results into
n_trees = []
r2_list = []
rmse_list = []
pRMSE_list = []
variance_list = []
Bias_list = []
n_samples =[]


# file pattern to search for in the directory.
pattern = "%sPredictedANDobserved_*" % direc

# return a list of files matching the pattern "csv" in the directry 
file1 = glob.glob(pattern)

for path in file1:
    a = path[2:]
    filelist.append(a)

# loop through the list of csv files and opens them as dataframes and selectes the rmse results and transforms them and returns them to a list
for filename in filelist:


    # import the raw results and calculate the statstics
    n_t = int(filename[22:-4])
    n_trees.append(n_t)
    dfa = pd.read_csv(filename, header=None)# reads the csv file
    # sort out the outputs into two colums of data to calculate the stats
    pred =  dfa[dfa.columns[0:20]]
    obs = dfa[dfa.columns[20:]]
    
    # stack the results of the 20 runs on top of each other 
    st_pred = pred.stack()
    st_obs = obs.stack()
    
    a = st_pred.values
    b = st_obs.values
    
    df = pd.DataFrame([a,b]).T
    df.columns=['predicted','observed']
    
    # selects the model using the spectral predictor variables 
    y = df.observed.values
    x = df.predicted.values
    df['res_L5'] = df['observed'] - df['predicted']
    sumRes = df['res_L5'].sum()
    """
    code to calculate the pearson's correlation coefficient and the coefficient of determination, 
    the for loop extracts out the correlatin coeffiecent score which is a tuple containing the R and p value 
    """
    r = pearsonr(x,y)
    r2 = r[0:1]
    personsr = []
    for i in r:
        personsr.append(i)
    r2 = personsr[0]**2
    r2s = str(r2)

    # calculates the variance 
    meanN13error = df['res_L5'].mean()
    n1 = len(df['predicted'])-1 
    df['var1_l5'] = (df['res_L5']-meanN13error)**2
    vari_n13 = str(df['var1_l5'].sum()/n1)

    # formats the stats into strings to enable them to be put in as text in the plot
    r2s = str(format(r2, '.2f'))
    r2_list.append(r2s)
    rmseS = str(format(np.sqrt(np.mean((y - x) ** 2)), '.3f'))
    rmse_list.append(rmseS)
    var_n13 = str(format(df['var1_l5'].sum()/n1,'.3f'))
    variance_list.append(var_n13)
    biasS = str(format(sumRes /len(y), '.3f'))
    Bias_list.append(biasS)
    nS = str(len(x))
    n_samples.append(nS)
    # calculate the RMSE expressed as a percentage of the observed mean (RMSE%) for each response variable.

    rRMSE = str(format((np.sqrt(np.mean((y - x) ** 2))/np.mean(y))*100,'.2f'))
    pRMSE_list.append(rRMSE)
    
# create a data frame with the results and export to csv
df_res = pd.DataFrame([n_trees,r2_list,rmse_list, pRMSE_list,variance_list, Bias_list, n_samples]).T
df_res.columns= ['n_trees', 'R2', 'RMSE', 'RMSE%','Variance','Bias','n_samples']
clean_res = df_res.sort_values(by=['n_trees'])
clean_res.to_csv("l8_30m_n_trees_results_" + hpar + "_CanopyHeight.csv")

In [5]:
#results of the n trees analysis 
clean_res

,n_trees,R2,RMSE,RMSE%,Variance,Bias,n_samples
0,512,0.75,3.154,23.40,9.947,-0.026,296000


In [6]:
# read in the selected n_trees band importance scores list
impS = pd.read_csv('FeatureImportance_Band_Score_n512.csv',header=0)
print (shape(impS))

(56, 6)


In [7]:
list_var = (impS['band'].values)
print (len(list_var))
list_var

56


array(['CVId', 'ratio63d', 'ratio63a', 'ratio52a', 'psB5a', 'psB5d',
       'ratio52d', 'ratio53d', 'psB6d', 'NDVId', 'ratio54d', 'ratio53a',
       'ratio43d', 'NDIId', 'MSRd', 'NDIIa', 'ratio54a', 'GNDVId',
       'ratio42d', 'CVIa', 'ratio43a', 'psB2a', 'MSRa', 'NDVIa',
       'ratio62a', 'psB2d', 'psB6a', 'ratio62d', 'psB3a', 'ratio64d',
       'NBRd', 'psB3d', 'RId', 'ratio32d', 'NDGId', 'ratio65d', 'ratio65a',
       'ratio42a', 'GNDVIa', 'NBRa', 'ratio64a', 'ratio32a', 'RIa',
       'NDGIa', 'GSAVIa', 'psB4a', 'GDVIa', 'MSAVId', 'psB4d', 'SAVId',
       'SAVIa', 'GSAVId', 'MSAVIa', 'GDVId', 'DVIa', 'DVId'], dtype=object)

In [8]:
conv_list = list_var.tolist()
variables2 = conv_list
print (len(variables2))

56


In [9]:
print (variables2)

['CVId', 'ratio63d', 'ratio63a', 'ratio52a', 'psB5a', 'psB5d', 'ratio52d', 'ratio53d', 'psB6d', 'NDVId', 'ratio54d', 'ratio53a', 'ratio43d', 'NDIId', 'MSRd', 'NDIIa', 'ratio54a', 'GNDVId', 'ratio42d', 'CVIa', 'ratio43a', 'psB2a', 'MSRa', 'NDVIa', 'ratio62a', 'psB2d', 'psB6a', 'ratio62d', 'psB3a', 'ratio64d', 'NBRd', 'psB3d', 'RId', 'ratio32d', 'NDGId', 'ratio65d', 'ratio65a', 'ratio42a', 'GNDVIa', 'NBRa', 'ratio64a', 'ratio32a', 'RIa', 'NDGIa', 'GSAVIa', 'psB4a', 'GDVIa', 'MSAVId', 'psB4d', 'SAVId', 'SAVIa', 'GSAVId', 'MSAVIa', 'GDVId', 'DVIa', 'DVId']


In [10]:
# iterate over the list and remove the bands that are transformatons of themselves based on the order of importance

In [11]:
for var in variables2:
    #print (var)
    if var in variables2:
        
        # 'GNDVId''ratio42d'
        if var == 'GNDVId':
            variables2.remove('ratio42d')
    
        elif var == 'ratio42d':
                variables2.remove('GNDVId')

        ##################################################
        ##################################################            
            
        # 'GNDVIa''ratio42a'
        if var == 'GNDVIa':
            variables2.remove('ratio42a')
    
        elif var == 'ratio42a':
                variables2.remove('GNDVIa')

        ##################################################
        ##################################################    


        # 'ratio43a', 'NDVIa',MSRa      
        elif var == 'ratio43a':
                variables2.remove('NDVIa')
                variables2.remove('MSRa')

        elif var == 'NDVIa':
                variables2.remove('ratio43a')
                variables2.remove('MSRa')  

        elif var == 'MSRa':
                    variables2.remove('ratio43a')
                    variables2.remove('NDVIa')
       
        ##################################################
        ##################################################            

        # 'NDGId', 'ratio32d',RId            
        elif var == 'NDGId':
                variables2.remove('ratio32d')
                variables2.remove('RId') 
      
        elif var == 'ratio32d':
                variables2.remove('NDGId')
                variables2.remove('RId')
                     

        elif var == 'RId':
                variables2.remove('ratio32d')
                variables2.remove('NDGId')

        ##################################################
        ##################################################            

        # 'NDGIa','RIa',ratio32a           
        elif var == 'NDGIa':
                variables2.remove('ratio32a')
                variables2.remove('RIa')
       
        elif var == 'ratio32a':
                variables2.remove('NDGIa')
                variables2.remove('RIa')

        elif var == 'RIa':
                variables2.remove('ratio32a')
                variables2.remove('NDGIa') 

        ##################################################
        ################################################## 

        #'MSRd','ratio43d',NDVId

        elif var == 'ratio43d':
                variables2.remove('NDVId')
                variables2.remove('MSRd')

        elif var == 'NDVId':
                variables2.remove('ratio43d')
                variables2.remove('MSRd')

        elif var == 'MSRd':
                variables2.remove('ratio43d')
                variables2.remove('NDVId')

        ##################################################
        ################################################## 
                   
        # 'NBRa', ratio104a  

        elif var == 'NBRa':
                variables2.remove('ratio64a')

        elif var == 'ratio64a':
                variables2.remove('NBRa')

        ##################################################
        ##################################################             

        # 'ratio104d',NBRd  

        elif var == 'NBRd':
                variables2.remove('ratio64d')

        elif var == 'ratio64d':
                variables2.remove('NBRd')            

        ##################################################
        ##################################################     

        # 'NDIIa', - ratio54a

        elif var == 'NDIIa':
                variables2.remove('ratio54a')

        elif var == 'ratio54a':
                variables2.remove('NDIIa')            

        ##################################################
        ################################################## 
        
        # 'NDIIa', - ratio54a

        elif var == 'NDIId':
                variables2.remove('ratio54d')

        elif var == 'ratio54d':
                variables2.remove('NDIId')            

        ##################################################
        ################################################## 
                
print (len(variables2))
print (variables2)

42
['CVId', 'ratio63d', 'ratio63a', 'ratio52a', 'psB5a', 'psB5d', 'ratio52d', 'ratio53d', 'psB6d', 'NDVId', 'ratio54d', 'ratio53a', 'NDIIa', 'GNDVId', 'CVIa', 'ratio43a', 'psB2a', 'ratio62a', 'psB2d', 'psB6a', 'ratio62d', 'psB3a', 'ratio64d', 'psB3d', 'RId', 'ratio65d', 'ratio65a', 'ratio42a', 'NBRa', 'ratio32a', 'GSAVIa', 'psB4a', 'GDVIa', 'MSAVId', 'psB4d', 'SAVId', 'SAVIa', 'GSAVId', 'MSAVIa', 'GDVId', 'DVIa', 'DVId']


In [12]:
# read in the training data to assess the relationship between the top 30 variables 
df = pd.read_csv('C:/Users/grants/code/old_growth/sen2_l8/l8_combined/l8_model_dev/stage01_02/combined_training_data_l8_int_version_n148000.csv', header=0)
print (shape(df))

(148000, 77)


In [13]:
pred_vars = df[variables2]

pred_vars.to_csv('predict_vars_correlation.csv')

In [14]:
print (pred_vars.shape)
print (list(pred_vars))

(148000, 42)
['CVId', 'ratio63d', 'ratio63a', 'ratio52a', 'psB5a', 'psB5d', 'ratio52d', 'ratio53d', 'psB6d', 'NDVId', 'ratio54d', 'ratio53a', 'NDIIa', 'GNDVId', 'CVIa', 'ratio43a', 'psB2a', 'ratio62a', 'psB2d', 'psB6a', 'ratio62d', 'psB3a', 'ratio64d', 'psB3d', 'RId', 'ratio65d', 'ratio65a', 'ratio42a', 'NBRa', 'ratio32a', 'GSAVIa', 'psB4a', 'GDVIa', 'MSAVId', 'psB4d', 'SAVId', 'SAVIa', 'GSAVId', 'MSAVIa', 'GDVId', 'DVIa', 'DVId']


In [15]:
import pandas as pd
import os

def corre(data,n):
    """
    This code performs the calculations and sorts the results. 
    """
    cor = data.corr().abs() # pearsons correlation coefficient
    cus = cor.unstack() # sort the data
    r_sort = cus.sort_values(kind="quicksort",ascending=False) # sort the data in accending order based on the highest r2 value
       
    return r_sort

def read_Data():
    """
    This reads in the csv file containing the test and train data and selects the colums for analysis.
    """
    df  = pd.read_csv('predict_vars_correlation.csv', header=0)
    df.dropna()
    data = df[df.columns[1:]]
    n = len(data.columns)
    
    return data,n
    
def mainRoutine():
    
    data,n = read_Data()
   
    r_sort = corre(data,n) 
    
    r_sort.to_csv('temp.csv')
    
    df = pd.read_csv('temp.csv',header=None)
    
    df.columns = ['b1','b2','corr'] # give the df headers
    
    df2 = df[df['b1'] != df['b2']] # remove all the self correlated data by comparing the band names in the two columns
      
    df2.to_csv('Var_corr_results_L5_all_vars.csv') # reads out the results with new header names
    
    os.remove('temp.csv') # delete the temp csv file
    
if __name__ == "__main__":
    mainRoutine()   

In [16]:
df2 = pd.read_csv('Var_corr_results_L5_all_vars.csv',header=0)
print (list(df2))
print (shape(df2))

['Unnamed: 0', 'b1', 'b2', 'corr']
(1722, 4)


In [17]:
# find all the highly correlated variables and get a list to produce a scatter matix 

h_corr = df2[df2['corr'] >= .95] #finds all correlation values % or greater
h_corr_list = h_corr['b1'].unique() # returns a list of the bands and prints them. 

newlist = []

for i in h_corr_list:
    newlist.append(i)
print (newlist)

['SAVId', 'SAVIa', 'MSAVIa', 'DVIa', 'DVId', 'MSAVId', 'GSAVIa', 'GDVIa', 'psB5a', 'psB5d', 'psB3d', 'psB3a', 'GDVId', 'GSAVId', 'psB6a', 'psB6d', 'NBRa', 'NDIIa', 'psB2a', 'psB2d', 'ratio54d', 'ratio64d']


In [18]:
# get a copy of the predictor variables after the band transformations have been removed

vari = variables2
print (len(vari))
print (list(vari))

42
['CVId', 'ratio63d', 'ratio63a', 'ratio52a', 'psB5a', 'psB5d', 'ratio52d', 'ratio53d', 'psB6d', 'NDVId', 'ratio54d', 'ratio53a', 'NDIIa', 'GNDVId', 'CVIa', 'ratio43a', 'psB2a', 'ratio62a', 'psB2d', 'psB6a', 'ratio62d', 'psB3a', 'ratio64d', 'psB3d', 'RId', 'ratio65d', 'ratio65a', 'ratio42a', 'NBRa', 'ratio32a', 'GSAVIa', 'psB4a', 'GDVIa', 'MSAVId', 'psB4d', 'SAVId', 'SAVIa', 'GSAVId', 'MSAVIa', 'GDVId', 'DVIa', 'DVId']


In [19]:
"""
this block of code iterates over the list of predictor variables in order of importance
and removes the highly correlated variables to output as a new df

"""
removed_variables = []

for v in vari:
    # take the variable in order of importance and query the correlation assessment 
    # data frame and identify the variables that are highly correlated
    data3 = h_corr[h_corr['b1'] == v] 

    
    # convert the highly correlated variables into a list to enable 
    # them to be iterated over and removed from the vari2 list
    
    values = (data3['b2'].values)
    values_list = [values]
    
    for varCom in values_list:
        
        # iterate over the new list of variables and remove them from the vari list
        
        for var in varCom:
                        
            if var in vari:
                removed_variables.append(var)
                vari.remove(var)
                
print (len(vari))                
print (vari)

30
['CVId', 'ratio63d', 'ratio63a', 'ratio52a', 'psB5a', 'ratio52d', 'ratio53d', 'psB6d', 'NDVId', 'ratio54d', 'ratio53a', 'NDIIa', 'GNDVId', 'CVIa', 'ratio43a', 'psB2a', 'ratio62a', 'ratio62d', 'psB3d', 'RId', 'ratio65d', 'ratio65a', 'ratio42a', 'ratio32a', 'GSAVIa', 'psB4a', 'MSAVId', 'psB4d', 'SAVId', 'GSAVId']


In [20]:
final_list = vari
final_list.insert(0,hpar2)

In [21]:
print (list(final_list))

['perc99_1', 'CVId', 'ratio63d', 'ratio63a', 'ratio52a', 'psB5a', 'ratio52d', 'ratio53d', 'psB6d', 'NDVId', 'ratio54d', 'ratio53a', 'NDIIa', 'GNDVId', 'CVIa', 'ratio43a', 'psB2a', 'ratio62a', 'ratio62d', 'psB3d', 'RId', 'ratio65d', 'ratio65a', 'ratio42a', 'ratio32a', 'GSAVIa', 'psB4a', 'MSAVId', 'psB4d', 'SAVId', 'GSAVId']


In [22]:
final_list2 = df[final_list]
print (len(list(final_list2)))

31


In [23]:
final_list2.to_csv("l8_30m_" + hpar + "_pred_vars_stage2_allvars.csv")

In [24]:
print (removed_variables)
print (len(removed_variables))

['psB5d', 'psB6a', 'ratio64d', 'NBRa', 'psB2d', 'psB3a', 'GDVIa', 'DVIa', 'DVId', 'SAVIa', 'MSAVIa', 'GDVId']
12
